<a href="https://colab.research.google.com/github/germanbraun/cursoKG/blob/main/Labs/RDFlib/APIToRDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar dependencias

In [13]:
!pip install rdflib
!pip install SPARQLWrapper
!pip install requests

Importar py librerías

In [14]:
import urllib.parse
import rdflib
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import FOAF, RDF

from SPARQLWrapper import SPARQLWrapper, BASIC, POST

import requests
import logging
import json
from json.encoder import JSONEncoder

Consultar una REST API y convertir a RDF

In [19]:
try:
    result = requests.get(url = "https://reqres.in/api/users", timeout=2 )
    import time
    time.sleep(2)
    if result.ok :
        responseJSON = result.json()
        print(responseJSON)
        namedGraph = Graph()
        namedGraph.bind("foaf", FOAF)

        # connect to GraphDB
        #sparql = SPARQLWrapper("http://127.0.0.1:7200/repositories/curso-kg/statements")
        #sparql.setMethod(POST)

        # create named graph
        for entry in responseJSON['data']:
            url_encode = urllib.parse.quote(str(entry['id']))
            person = URIRef(f'http://example.org/people/{url_encode}')
            firstName = Literal(entry['first_name'])
            lastName = Literal(entry['last_name'])
            mailbox = Literal(entry['email'])
            thumbnail = Literal(entry['avatar'])

            namedGraph.add((person, RDF.type, FOAF.Person))
            namedGraph.add((person, FOAF.firstName, firstName))
            namedGraph.add((person, FOAF.lastName, lastName))
            namedGraph.add((person, FOAF.mbox, mailbox))
            namedGraph.add((person, FOAF.thumbnail, thumbnail))

            for s, p, o in namedGraph:
                if o.__class__ == rdflib.term.Literal:
                    query = f"""
                        INSERT DATA {{
                            GRAPH <http://example.org/toy/graph> {{
                                <{s}> <{p}> "{o}"
                            }}
                        }}
                    """
                else:
                    query = f"""
                        INSERT DATA {{
                            GRAPH <http://example.org/toy/graph> {{
                                <{s}> <{p}> <{o}>
                            }}
                        }}
                """
                logging.info(query)
                print(query)
                #sparql.setQuery(query)
                #sparql.query()
    else:
        Exception("Request returns empty")

except Exception as e:
    logging.error(F"Error when querying the API: {e}")

{'page': 1, 'per_page': 6, 'total': 12, 'total_pages': 2, 'data': [{'id': 1, 'email': 'george.bluth@reqres.in', 'first_name': 'George', 'last_name': 'Bluth', 'avatar': 'https://reqres.in/img/faces/1-image.jpg'}, {'id': 2, 'email': 'janet.weaver@reqres.in', 'first_name': 'Janet', 'last_name': 'Weaver', 'avatar': 'https://reqres.in/img/faces/2-image.jpg'}, {'id': 3, 'email': 'emma.wong@reqres.in', 'first_name': 'Emma', 'last_name': 'Wong', 'avatar': 'https://reqres.in/img/faces/3-image.jpg'}, {'id': 4, 'email': 'eve.holt@reqres.in', 'first_name': 'Eve', 'last_name': 'Holt', 'avatar': 'https://reqres.in/img/faces/4-image.jpg'}, {'id': 5, 'email': 'charles.morris@reqres.in', 'first_name': 'Charles', 'last_name': 'Morris', 'avatar': 'https://reqres.in/img/faces/5-image.jpg'}, {'id': 6, 'email': 'tracey.ramos@reqres.in', 'first_name': 'Tracey', 'last_name': 'Ramos', 'avatar': 'https://reqres.in/img/faces/6-image.jpg'}], 'support': {'url': 'https://reqres.in/#support-heading', 'text': 'To kee